# Content of the notebook

This notebook contains all steps that were used to create the training, validation, and test set, which we used in the paper "Prediction of Michaelis constants $K_M$ from structural features".

We note that the BRENDA database is constantly changing and therefore, the dataset that is created with this notebook will be different to the dataset that we used in our paper. This notebook is divided into five steps. After each of the steps 1. to 4., we saved our dataset and stored it in the folder for datasets (named "brenda_df_checkpoint_01.pkl" - "brenda_df_checkpoint_04.pkl"). We also stored our final training, validation, and test sets in this folder (named "test_data.pkl", "training_data.pkl", and "validation_data.pkl"). All datasets that are newly created with this notebook, will be stored with the same filenames plus the suffix "_V2".

The location of the folder with all datasets must be changed (to the corresponding path on your PC) in the python file "directory_infomation.py" before executing this notebook.

## The notebook consists of the following steps:

1. Downloading data from BRENDA
2. Mapping data points to KEGG reaction IDs
3. Downloading functional domains for amino acid sequences of the enzymes
4. Adding extended-connectivity fingerprints (ECFPs) and extra features (LogP, MW) to the dataset
5. Splitting the dataset in training, validation, and test set

Loading all the necessary packages and functions to execute the code:

In [1]:
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from rdkit import Chem
from rdkit.Chem import Crippen
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem
import urllib.request
from zeep import Client
import hashlib
import warnings
warnings.filterwarnings('ignore')

from functions_for_data_preprocessing import *
from directory_infomation import *

## 1. Downloading data from BRENDA

Downloading data from BRENDA can take a couple of hours.

### (a) Downloading $K_M$ values together with substrate name, EC number and organism from BRENDA

To download the data from BRENDA, a registration is needed (https://www.brenda-enzymes.org/register.php). In the following cell the password and email address has to be stored:

In [2]:
wsdl = "https://www.brenda-enzymes.org/soap/brenda_zeep.wsdl"
password = hashlib.sha256("a2b8c6".encode("utf-8")).hexdigest()
email = "alexander.kroll@hhu.de"
client = Client(wsdl)

Creating a list with all EC numbers that exist in BRENDA:

In [3]:
parameters = (email,password)
EC_numbers = client.service.getEcNumbersFromEcNumber(*parameters)
print("There exist %s different EC numbers in the BRENDA database." % len(EC_numbers))

There exist 8083 different EC numbers in the BRENDA database.


Iterating over all EC numbers and downloading all existing Km value together with the organism name, substrate name, literature ID and a comment (the amino acid sequence of the enzyme has to be downloaded separately afterwards). 

In [4]:
# create empty pandas DataFrame
brenda_df = pd.DataFrame(columns= ["EnzymeType", "ecNumber", "KM", "sequence",
                       "organism", "substrate", "commentary"])

for ind in range(len(EC_numbers)):
    EC = EC_numbers[ind]
    #If you want to keep track of the progress, you can print the EC number that is currently progressed:
    #print(EC)
    
    #set parameters for the next download
    parameters = (email, password, "ecNumber*"+EC, "kmValue*", "kmValueMaximum*", "substrate*",
                  "commentary*", "organism*", "ligandStructureId*", "literature*")
    resultString = download_data_from_BRENDA(function = client.service.getKmValue, parameters = parameters,
                                        error_identifier = EC, print_error= False)
    
    #save all necessary information in the DataFrame brenda_df
    brenda_df = extract_KM_info_from_resultString(brenda_df, resultString)

The comments, that are downloaded together with the KM values, can contain information about the type of the enzyme (wild type, mutant, recombinant). Where possible, we extract this information and store it in the column "EnzymeType". We also remove duplicated data points:

In [5]:
brenda_df.loc[pd.isnull(brenda_df["commentary"])] = ""
brenda_df["EnzymeType"][brenda_df['commentary'].str.contains("wild")] = "wild type"
brenda_df["EnzymeType"][brenda_df['commentary'].str.contains("mutant")] = "mutant"
brenda_df["EnzymeType"][brenda_df['commentary'].str.contains("recombin")] = "recombinant"
brenda_df.drop(columns = ["commentary"], inplace = True)
brend_df = brenda_df.drop_duplicates(keep="first").reset_index(drop=True)

### (b) Downloading amino acid sequence from BRENDA for every EC number-organism combination:


(i) Extracting all EC number-organism combinations and store them in a DataFrame:

In [6]:
EC_org_df = pd.DataFrame(data = {"ecNumber" : brenda_df["ecNumber"], "organism" : brenda_df["organism"]})
EC_org_df = EC_org_df.drop_duplicates(keep = "first").reset_index(drop = True)
print("There exist %s different EC number-organism combinations in the DataFrame." % len(EC_org_df))

There exist 16633 different EC number-organism combinations in the DataFrame.


(ii) Downloading the enzyme sequences from BRENDA:

In [7]:
EC_org_df["sequence"] = np.nan

for ind in EC_org_df.index:
    org = EC_org_df["organism"][ind]
    EC = EC_org_df["ecNumber"][ind]
    if not pd.isnull(EC) and not pd.isnull(org):
        parameters = ("alexander.kroll@hhu.de",password,"ecNumber*" + str(EC), "sequence*", "noOfAminoAcids*",
                      "firstAccessionCode*", "source*", "id*", "organism*" + str(org))

        resultString = download_data_from_BRENDA(function = client.service.getSequence,
                                            parameters = parameters, error_identifier = EC +";" +org,
                                           print_error = False)
        if resultString != [] and resultString is not None:
            EC_org_df["sequence"][ind] = resultString[0]["sequence"]

        #save a backup after 1000 downloads:     
        if ind % 1000 == 0:
            #print(ind)
            EC_org_df.to_csv(datasets_dir + "EC_and_org_with_seq_V2.csv")

Removing all data points without an amino acid sequence:

In [8]:
brenda_df = brenda_df.loc[~pd.isnull(brenda_df["sequence"])]

Saving the created dataset:

In [9]:
#brenda_df.to_pickle(datasets_dir + "brenda_df_checkpoint_1_V2.pkl")

Alternatively to executing all the steps above, our dataset can be loaded:

In [10]:
brenda_df = pd.read_pickle(datasets_dir + "brenda_df_checkpoint_1.pkl")

## 2. Mapping data points from BRENDA to KEGG reaction IDs

### (a) Creating a DataFrame with all KEGG Compound IDs and the corresponding compound names. Data to create this DataFrame is downloaded from the KEGG homepage:

To create the described DataFrame, we download data from KEGG, which can take a couple of hours. The resulting DataFrame is stored in the directory for datasets and is named "KEGG_substrate_df.pkl". To newly create this DataFrame, execute the following cell. Otherwise, load the DataFrame by executing the cell after this one.

In [11]:
#create_df_with_KEGG_CIDs_and_correpsonding_names()

Alternatively to creating a new DataFrame with the above cell, our existing DataFrame can be used:

In [12]:
KEGG_substrate_df = pd.read_pickle(datasets_dir + "KEGG_substrate_df.pkl")
KEGG_substrate_df.head()

,KEGG ID,substrate
0,C00001,H2O
1,C00001,Water
2,C00002,ATP
3,C00002,Adenosine 5'-triphosphate
4,C00003,NAD+


### (b) Mapping substrate names from BRENDA to KEGG Compound IDs (CIDs) with the DataFrame created/loaded in (a)

Converting all substrate names in KEGG_substrate_df and brenda_df to substrate names that only contain lower case letters:

In [13]:
KEGG_substrate_df["substrate"] = [name.lower() for name in KEGG_substrate_df["substrate"]]
brenda_df["substrate"] = [name.lower() for name in brenda_df["substrate"]]

Mapping substrate names from brenda_df to KEGG CIDs:

In [14]:
brenda_df = brenda_df.merge(KEGG_substrate_df, on = "substrate", how = "left")
print("For %s data points we could not map the substrate name to a KEGG Compound ID." % sum(pd.isnull(brenda_df["KEGG ID"])))

For 13798 data points we could not map the substrate name to a KEGG Compound ID.


###  (c) For all the substrate names that could not be mapped to KEGG CIDs yet: We try to map them with a synonym list from PubChem to PubChem CIDs first and then to KEGG CIDs

The synonym list from PubChem can be downloaded with the following link: ftp.ncbi.nlm.nih.gov/pubchem/Compound/Extras/CID-Synonym-filtered.gz and should be stored in the folder for datasets.

We split this list with over 12 million entries in five parts because as a single file it is too big to be stored in our RAM:

In [15]:
#divivde synonym database in 5 diffeerent databases
for k in range(5):
    count = 0
    cid = []
    substrates = []
    with open(datasets_dir + "/CID-Synonym-filtered") as infile:
        if count % 10**6 == 0:
        for line in infile:
            count +=1
            if not (count < (k+1)*25632777 and count >=  k*25632777): 
                None
            else:
                line = line.split("\t")
                cid.append(line[0])
                substrates.append(line[1].lower()[0:-1])
                
    df = pd.DataFrame(data = {"substrates": substrates, "CID" :cid})
    df.to_pickle("./substrates_synonyms_part"+str(k)+".pkl")

Mapping all substrate names without a KEGG CID to PubChem CIDs:

In [16]:
#creating a list with all substrate names that couldn't be mapped to KEGG IDs
unmapped_substrates = list(set(list(brenda_df["substrate"][pd.isnull(list(brenda_df["KEGG ID"]))])))
matches = substrate_names_to_Pubchem_CIDs(unmapped_substrates)

loading part 1 of 5 of the synonym list...
searching in synoynm list part 1 for matches
loading part 2 of 5 of the synonym list...
searching in synoynm list part 2 for matches
loading part 3 of 5 of the synonym list...
searching in synoynm list part 3 for matches
loading part 4 of 5 of the synonym list...
searching in synoynm list part 4 for matches
loading part 5 of 5 of the synonym list...
searching in synoynm list part 5 for matches


Mapping all PubChem CIDs to KEGG CIDs: Therefore, we create a txt-file ("Pubchem_CIDs.txt") that contains all PubChem CIDs that we want to map to KEGG CIDs:

In [17]:
#convert CIDs from strings to integers:
for ind in matches.index:
    cid = matches.loc[ind]["CID"]
    if not pd.isnull(cid):
        matches["CID"][ind] = int(cid)
        
        
#create txt file with all CIDs in matches:
CIDs = list(matches.loc[~ pd.isnull(matches["CID"])]["CID"])
f = open(datasets_dir + "Pubchem_CIDs.txt","w") 
for cid in CIDs:
    f.write(str(cid) + "\n")
f.close()

The txt-file "Pubchem_CIDs.txt" can be used as the input for the webservice http://csbg.cnb.csic.es/mbrole2/conversion.php to map the Pubchem CIDs to KEGG CIDs. The resulting file "mbrole_conversion.tsv" should be stored in the directory for datasets.

In [22]:
macthes_with_KEGG_IDs

,Metabolite,CID,Input_source,KEGG ID,Output_source
0,"n,2-diphenylacetamide",225533,NaN,NaN,NaN
1,7-ethyl-10-(4-(1-piperidino)-1-piperidino)carb...,NaN,NaN,NaN,NaN
2,"n-acetyl-s-(e,e)-farnesyl-d-cysteine",129661286,NaN,NaN,NaN
3,(s)-2-aminophenylacetamide,NaN,NaN,NaN,NaN
4,(2s)-methylmalonyl-coenzyme a,NaN,NaN,NaN,NaN
...,...,...,...,...,...
7102,(s)-propanolol,NaN,NaN,NaN,NaN
7103,"5,10-methylene-5,6,7,8-tetrahydrofolic acid",NaN,NaN,NaN,NaN
7104,unmodified e. coli trna1val,NaN,NaN,NaN,NaN
7105,dansyl-gngn,NaN,NaN,NaN,NaN


In [23]:
#load the resulting file and store it in a DataFrame:
Pubchem_CID_to_KEGG_df = pd.read_csv(datasets_dir + "mbrole_conversion.tsv", sep= "\t")
#rename columns:
Pubchem_CID_to_KEGG_df.rename(columns = {"Input" : "CID", "Output" : "KEGG ID"}, inplace = True)

#merge this DataFrame with the DataFrame called matches:
macthes_with_KEGG_IDs = pd.merge(matches, Pubchem_CID_to_KEGG_df, how='left', on=['CID'])
macthes_with_KEGG_IDs.drop(columns = ["Input_source", "Output_source"], inplace = True)
macthes_with_KEGG_IDs.rename(columns = {"Metabolite" : "substrate"}, inplace = True)

macthes_with_KEGG_IDs.head()

,substrate,CID,KEGG ID
0,"n,2-diphenylacetamide",225533,NaN
1,7-ethyl-10-(4-(1-piperidino)-1-piperidino)carb...,NaN,NaN
2,"n-acetyl-s-(e,e)-farnesyl-d-cysteine",129661286,NaN
3,(s)-2-aminophenylacetamide,NaN,NaN
4,(2s)-methylmalonyl-coenzyme a,NaN,NaN


Mapping all newly found KEGG CIDs for substrate names in the Brenda DataFrame:

In [24]:
for ind in brenda_df.index:
    if pd.isnull(brenda_df["KEGG ID"][ind]):
        substrate = brenda_df["substrate"][ind]
        try:
            KEGG_ID = list(macthes_with_KEGG_IDs.loc[macthes_with_KEGG_IDs["substrate"]== substrate]["KEGG ID"])[0]
            brenda_df["KEGG ID"][ind] = KEGG_ID
        except:
            None

Removing all data points with substrates without a KEGG CID:

In [25]:
brenda_df = brenda_df.loc[~pd.isnull(brenda_df["KEGG ID"])]

### (d) Mapping  data points to KEGG reaction IDs

Loading DataFrame with all reactions in the KEGG reaction database:

In [26]:
KEGG_reaction_df = pd.read_pickle(datasets_dir + "KEGG_reaction_df.pkl")
KEGG_reaction_df.head()

,EC number,KEGG reaction ID,substrates left,substrates right,KEGG IDs left,KEGG IDs right
0,1.1.1.1,R00623,Primary alcohol + NAD+,Aldehyde + NADH + H+,"[C00226, C00003]","[C00071, C00004, C00080]"
1,1.1.1.1,R00624,Secondary alcohol + NAD+,Ketone + NADH + H+,"[C01612, C00003]","[C01450, C00004, C00080]"
2,1.1.1.1,R00754,Ethanol + NAD+,Acetaldehyde + NADH + H+,"[C00469, C00003]","[C00084, C00004, C00080]"
3,1.1.1.1,R02124,Retinol + NAD+,Retinal + NADH + H+,"[C00473, C00003]","[C00376, C00004, C00080]"
4,1.1.1.1,R02878,1-Octanol + NAD+,1-Octanal + NADH + H+,"[C00756, C00003]","[C01545, C00004, C00080]"


Mapping the BRENDA data points to KEGG reaction IDs via EC number and KEGG CID:

In [30]:
brenda_df["KEGG reaction ID"] = np.nan
for ind in brenda_df.index:
    reaction_ids = map_BRENDA_entry_to_KEGG_reaction_ID(brenda_df.loc[ind], KEGG_reaction_df)
    if not reaction_ids is None:
        brenda_df["KEGG reaction ID"][ind] = reaction_ids

Removing all data points without a KEGG reaction ID:

In [31]:
brenda_df = brenda_df.loc[~pd.isnull(brenda_df["KEGG reaction ID"])]

Saving the created DataFrame:

In [32]:
#brenda_df.to_pickle(datasets_dir + "brenda_df_checkpoint_2_V2.pkl")

Alternatively to executing all the steps above, our dataset can be loaded:

In [33]:
brenda_df = pd.read_pickle(datasets_dir + "brenda_df_checkpoint_2.pkl")

## 3. Downloading functional domains for emzyme amino acid sequences

In this part of the notebook, the functional domains for the amino acid sequences of the enzymes are downloaded via the Webservice of Pfam. There is an upper limit of 500 sequences per request. Therefore, we are saving the sequences in multiple FASTA-files of up to 500 sequences each.

First, extracting all different enzyme sequences and storing them in a DataFrame:

In [34]:
df_seq_FunD = pd.DataFrame(data = {"sequence" : brenda_df["sequence"]})
df_seq_FunD = df_seq_FunD.drop_duplicates(keep = "first").reset_index(drop = True)
df_seq_FunD["FunD"] = ""

Now, creating FASTA-files with the amino acid sequences of the enzymes:

In [35]:
part = 0
count = 0

ofile = open(datasets_dir + "Pfam_input_part_" + str(part) + ".txt", "w")
for ind in df_seq_FunD.index:
        if count > 499: # if 500 seq reached: use new file
            ofile.close()
            part +=1
            count = 0
            ofile = open(datasets_dir + "fasta_for_pfam_Km_part" + str(part) + ".txt", "w")
        seq = df_seq_FunD["sequence"][ind]
        if not pd.isnull(seq):
            count +=1
            seq_end = seq.find("#")
            ofile.write(">" + str(ind) + "\n" +seq[:seq_end] + "\n")
ofile.close()

These FASTA-files (named ""Pfam_input_part_k.txt") can be used as the input of the Pfam webservice under the following URL: https://pfam.xfam.org/search#tabview=tab1

Pfam sends the results of the requests via email. We copied the part of the email that begins with the Pfam output (beginning with ">") into a txt-file and saved it in the directory for datasets. We named the output files "pfam_output_partk.txt", where k is replaced by an integer according to the input file that was used to create this output file:

In [37]:
#FUND is a matrix where all different functional domains are stored
df_seq_FunD, FUND = calculate_FunD_vectors_from_Pfam_output(df = df_seq_FunD, no_of_parts = 16)

Only using functional domains that occur at least 5 times in the whole dataset:

In [38]:
selection = np.sum(FUND, axis= 0) >= 5
for ind in df_seq_FunD.index:
    if df_seq_FunD["FunD"][ind] != "":
        df_seq_FunD["FunD"][ind] = df_seq_FunD["FunD"][ind][selection]

Mapping functional domains to brenda_df:

In [39]:
brenda_df = brenda_df.merge(df_seq_FunD, on ="sequence", how = "left")

Saving the created DataFrame

In [40]:
#brenda_df.to_pickle(datasets_dir + "brenda_df_checkpoint_3_V2.pkl")

Alternatively to executing all the steps above, our dataset can be loaded:

In [41]:
brenda_df = pd.read_pickle(datasets_dir + "brenda_df_checkpoint_3.pkl")

## 4. Adding extended-connectivity fingerprints (ECFPs) and extra features (LogP, MW) to the dataset

ECFPs, molecular weight (MW) and LogP are calculated from an MDL Molfile of the substrate. The following function downloads Molfiles for all KEGG CIDs and stores them in the directory for datasets in the subfolder named "mol-files". This can take up to a couple of hours.

In [42]:
download_mol_files()

Folder for mol-files already exitsts. If you want to download all mol-files again, first remove the current folder.


Calculating the ECFP, MW and LogP for every substrate with an MDL Molfile as the input:

In [43]:
brenda_df["MW"] = np.nan
brenda_df["LogP"] = np.nan
brenda_df["ECFP"] = ""

for ind in brenda_df.index:
    kegg_id = brenda_df["KEGG ID"][ind]
    if not pd.isnull(kegg_id):
        try:
            mol = Chem.MolFromMolFile(datasets_dir +  "/mol-files/" + kegg_id + '.mol')
        except OSError:
            None
        if mol is not None:
            ecfp = AllChem.GetMorganFingerprintAsBitVect(mol, 3, nBits=1024)
            #convert fingerprint to 1024 binary numbers:
            ecfp = ecfp.ToBitString()
            brenda_df["ECFP"][ind] = str(ecfp) 
            brenda_df["MW"][ind] = Descriptors.ExactMolWt(mol)
            brenda_df["LogP"][ind] = Crippen.MolLogP(mol)

Saving the dataset:

In [44]:
#brenda_df.to_pickle(datasets_dir + "brenda_df_checkpoint_4_V2.pkl")

Alternatively to executing all the steps above, our dataset can be loaded:

In [45]:
brenda_df = pd.read_pickle(datasets_dir + "brenda_df_checkpoint_4.pkl")

## 5. Splitting the dataset in training, validation, and test set

Before splitting the data in training, test, and validation sets, we log10-transform the $K_M$ value and we take mean of these transformed values for data points with the same functional domains of the enzyme and the same KEGG CID for the substrate:

In [46]:
brenda_df["log10_Km"] = [np.log10(float(Km)) for Km in brenda_df["KM"]]

# To take the mean, we have to convert the functional domains into strings:
brenda_df["FunD"] = [str(list(FunD)) for FunD in brenda_df["FunD"]]

#taking the mean:
brenda_df = brenda_df.groupby(["KEGG ID", "FunD", "ECFP"]).mean().reset_index()

#we convert the functional domains back into arrays:
brenda_df["FunD"] = [eval(FunD) for FunD in brenda_df["FunD"]]
brenda_df

,KEGG ID,FunD,ECFP,KM,MW,LogP,log10_Km
0,C00002,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0000000001000000000000000000000000000000000000...,1.084803,506.995745,-1.6290,-0.917377
1,C00002,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0000000001000000000000000000000000000000000000...,0.025000,506.995745,-1.6290,-1.602060
2,C00002,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0000000001000000000000000000000000000000000000...,0.500000,506.995745,-1.6290,-0.301030
3,C00002,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0000000001000000000000000000000000000000000000...,0.420000,506.995745,-1.6290,-0.376751
4,C00002,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0000000001000000000000000000000000000000000000...,0.393333,506.995745,-1.6290,-0.447374
...,...,...,...,...,...,...,...
5153,C21895,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0000000000000000000000000000000001000000000000...,0.260000,168.042259,1.0060,-0.585027
5154,C21896,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0000000000000000000000000000000001000000000000...,0.019000,210.052823,1.2086,-1.721246
5155,C22127,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0000000000000100000000000100000101001000000000...,0.040000,622.444469,4.3248,-1.397940
5156,C22128,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0000000000000100000000100000000001001000000000...,0.311000,622.444469,4.3248,-0.507240


Splitting data in 70 % training data, 10% validation data, and 20 % test data:

In [47]:
y = np.array(brenda_df["log10_Km"])

#removing completely unrealsitc outliers:
brenda_df = brenda_df.loc[y> -5]
y = y[y> -5]

bins = np.linspace(min(y)+1.5, max(y)-1.5, 15)
y_binned = np.digitize(y, bins)
X_train, X_test, y_train, y_test = train_test_split(brenda_df, y_binned, test_size=0.2, stratify=y_binned, random_state=42)

y = np.array(X_train["log10_Km"])
bins = np.linspace(min(y)+1.5, max(y)-1.5, 15)
y_binned = np.digitize(y, bins)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_binned, test_size=0.125, stratify=y_binned, random_state=42)

In [48]:
X_test.to_pickle(datasets_dir + "test_data_V2.pkl")
X_train.to_pickle(datasets_dir + "training_data_V2.pkl")
X_val.to_pickle(datasets_dir + "validation_data_V2.pkl")